<a href="https://colab.research.google.com/github/ksolonenko/DLMath_assightment1/blob/main/Sentiment_analysis_using_RNN_emotion_dataset_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### IMDB Sentiment Analysis
This project uses a Recurrent Neural Network with LSTM using a sequence-to-sequence model to predict sentiment classification.  
Dataset can be found [here.]("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?resource=download")

In [1]:
!pip install keras_tuner
!pip install tensorflow[and-cuda]==2.16.1
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00
'import warnings' failed; traceback:
Traceback (most recent call last):
  File "/usr/lib/python3.10/warnings.py", line 538, in <module>
    _processoptions(sys.warnoptions)
  File "/usr/lib/python3.10/warnings.py", line 208, in _processoptions
    _setoption(arg)
  File "/usr/lib/python3.10/warnings.py", line 224, in _setoption
    import re
  File "/usr/lib/python3.10/re.py", line 127, in <module>
    import functools
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1012, in get_code
  File "<frozen importlib._bootstrap_external>", line 672, in _compile_bytecode
KeyboardInterrupt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

from datasets import load_dataset
from datasets import load_metric

import pandas as pd
import numpy as np
import tensorflow as tf

<ipython-input-2-e9b5659ed87e>:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [6]:
if tf.config.list_physical_devices('GPU'):
    #enable GPU
    tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

### Preprocessing data

In [7]:
dataset = load_dataset("dair-ai/emotion")

# convert to pandas DataFrame
train_df = dataset['train'].to_pandas()
validation_df = dataset['validation'].to_pandas()
test_df = dataset['test'].to_pandas()

dataset = pd.concat([train_df, validation_df, test_df], ignore_index=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: Interrupted by user

In [8]:
dataset.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [13]:
dataset['text'][0]
#to look at the general content and length of a text

'i didnt feel humiliated'

In [14]:
all_text = ''.join(dataset['text'].tolist())

unique_characters = set(all_text)
unique_characters = sorted(unique_characters)

print("Unique characters used in all strings:")
print(unique_characters)

Unique characters used in all strings:
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Text in the dataset only uses lowercase letters, so no further cleaning is required.

In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20000 non-null  object
 1   label   20000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


In [19]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [21]:
dataset["label"].value_counts()
#this dataset is a bit unbalanced with regards to items labeled with 5 (surprise)

label
1    6761
0    5797
3    2709
4    2373
2    1641
5     719
Name: count, dtype: int64

In [22]:
unique_labels = list(set(dataset['label']))
unique_labels

[0, 1, 2, 3, 4, 5]

The above encoded labels represent the following emotions:
- 0: sadness
- 1: joy
- 2: love
- 3: anger
- 4: fear
- 5: surprise

### One hot encoding

In [23]:
data_encoded = pd.get_dummies(dataset, columns=["label", ], dtype=int)

In [39]:
data_encoded.head()
#positive values are reresented by 1, and negative values are represented by 0

,text,label_0,label_1,label_2,label_3,label_4,label_5
0,i didnt feel humiliated,1,0,0,0,0,0
1,i can go from feeling so hopeless to so damned...,1,0,0,0,0,0
2,im grabbing a minute to post i feel greedy wrong,0,0,0,1,0,0
3,i am ever feeling nostalgic about the fireplac...,0,0,1,0,0,0
4,i am feeling grouchy,0,0,0,1,0,0


In [27]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data_encoded['text'])
X = pad_sequences(tokenizer.texts_to_sequences(data_encoded['text']), maxlen=300)  #padding to match train and test array

In [69]:
labels = data_encoded[['label_0', 'label_1', 'label_2', 'label_3', 'label_4', 'label_5']]
Y = np.array(labels)

### Splitting into training and testing data

In [67]:
train, test = train_test_split(data_encoded, test_size=0.2, random_state=24)
#only 2 columns in the data so no need to divide the dataset further into X and Y here

In [70]:
train["combined_labels"] = train[['label_0', 'label_1', 'label_2', 'label_3', 'label_4', 'label_5']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train["combined_labels"])

#word index dictionary
word_index = tokenizer.word_index

encoded_labels = tokenizer.texts_to_sequences(train["combined_labels"])

#pad encoded labels to a fixed length
max_length = max(len(x) for x in encoded_labels)
padded_labels = pad_sequences(encoded_labels, maxlen=max_length)

In [41]:
X_train.shape

(16000, 300)

In [42]:
X_test.shape

(4000, 300)

In [43]:
X_train
#array of tekenized text values

array([[   0,    0,    0, ...,  136, 1382,  934],
       [   0,    0,    0, ...,    2,   43,   12],
       [   0,    0,    0, ...,   64,   14,   84],
       ...,
       [   0,    0,    0, ...,  781,    7,  389],
       [   0,    0,    0, ...,  283,   26, 1815],
       [   0,    0,    0, ...,    1,   23, 3297]], dtype=int32)

In [44]:
Y_train

array([[0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

In [60]:
num_classes = len(unique_labels)

Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

In [61]:
Y_train

array([[[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       ...,

       [[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.

### Build model

Bidirectional LSTM layers were included instead of basic LSTM layers, as they can improve the model's ability to understand context in sequences by processing the input in both forward and backward directions.  To add complexity, L2 regularization was used to apply penalities on the Dense layer's kernel, and dropout layers were added to avoid overfitting. These additions will allow the model to learn more complex patterns.

Dropout rate values for hyperparameter tuning are build into the layers with values=[0.1, 0.2, 0.3], there are 4 dropout layers in total.

In [62]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=256))
    model.add(LSTM(128, dropout=hp.Choice('dropout_rate2', values=[0.1, 0.2, 0.3]), recurrent_dropout=0.20))
    model.add(LSTM(128, dropout=hp.Choice('dropout_rate2', values=[0.1, 0.2, 0.3]), recurrent_dropout=0.20))
    model.add(Dropout(hp.Choice('dropout_rate3', values=[0.1, 0.2, 0.3])))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01))) #dense layer with L2 regularization
    model.add(Dropout(hp.Choice('dropout_rate4', values=[0.1, 0.2, 0.3])))
    model.add(Dense(num_classes, activation='softmax')) #softmax chosen for multiclass classification

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

### Hyperparameter tuning

In [63]:
#initializing keras tuner
num_classes=len(unique_labels) #there are 6 classes

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='hyperparam_tuning'
)


Reloading Tuner from my_dir/hyperparam_tuning/tuner0.json


In [64]:
#format the data to suit Keras Tuner
def get_dataset(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=len(X)).batch(batch_size)
    return dataset

In [66]:
batch_sizes = [16, 32, 64]

for batch_size in batch_sizes:
    tuner.search(get_dataset(X_train, Y_train, batch_size), epochs=5, validation_data=get_dataset(X_test, Y_test, batch_size))

Epoch 1/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 122, 

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/py

In [ ]:
# Retrieve the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

In [ ]:
# Train the best model
history = best_model.fit(get_dataset(X_train, Y_train, best_hps.get('batch_size')),
                         epochs=5,
                         validation_data=get_dataset(X_test, Y_test, best_hps.get('batch_size')),
                         callbacks=[early_stopping, reduce_lr, model_checkpoint])


Epoch 1/50
 617/1250 ━━━━━━━━━━━━━━━━━━━━ 7:27 708ms/step - accuracy: 0.6681 - loss: 0.8469

KeyboardInterrupt: 

In [ ]:
model.save("sentiment_model.keras")

In [ ]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

### Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 69s 219ms/step - accuracy: 0.8911 - loss: 0.3163


In [ ]:
print(f'Accuracy: {accuracy}, Loss: {loss}')

Accuracy: 0.8916000127792358, Loss: 0.31192877888679504


### Predictions

In [ ]:
def sentiment_prediction(review):
    seq = tokenizer.texts_to_sequences([review])
    padded_seq = pad_sequences(seq, maxlen=300)

    prediction = model.predict(padded_seq)
    #prediction is a scole between 0 and 1, with 0 being a negative review and 1 being a positive review

    if prediction[0][0] > 0.6:
        sentiment = "This is a positive review."
    elif prediction[0][0] < 0.4:
        sentiment = "This is a negative review."
    else:
        sentiment = "This review is not strongly negative or positive."

    return sentiment

In [ ]:
sentiment_prediction("It was not my favourite movie") #this is an incorrect prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


'This is a positive review.'

In [ ]:
sentiment_prediction("I hated every minute of this movie")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


'This is a negative review.'

In [ ]:
sentiment_prediction("The costuming in this movie was great, and the art direction was some of my favourite")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


'This is a positive review.'

In [ ]:
sentiment_prediction("This movie is fantastic")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


'This is a positive review.'

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Save the best model
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model with callbacks
history = model.fit(X_train, Y_train,
                    validation_data=(X_test, Y_test),
                    epochs=50,
                    batch_size=16,
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])
